# CIFAR-10 Training Notebook
This notebook loads CIFAR-10, defines a simple CNN, trains it, evaluates on the test set, and saves the model weights to `models/cifar_net.pth`.

In [2]:
# Install required packages (uncomment if needed)
# !pip install torch torchvision matplotlib

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import os
from pathlib import Path

In [4]:
# Data transforms and loaders
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
# Define a simple CNN (matching the Streamlit app's Net)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

import torch.nn.functional as F
net = Net()

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net.to(device)

for epoch in range(25):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 200 == 199:    
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 200] loss: 2.305
[Epoch 1, Batch 400] loss: 2.300
[Epoch 1, Batch 400] loss: 2.300
[Epoch 1, Batch 600] loss: 2.291
[Epoch 1, Batch 600] loss: 2.291
[Epoch 2, Batch 200] loss: 2.187
[Epoch 2, Batch 200] loss: 2.187
[Epoch 2, Batch 400] loss: 2.126
[Epoch 2, Batch 400] loss: 2.126
[Epoch 2, Batch 600] loss: 2.037
[Epoch 2, Batch 600] loss: 2.037
[Epoch 3, Batch 200] loss: 1.912
[Epoch 3, Batch 200] loss: 1.912
[Epoch 3, Batch 400] loss: 1.856
[Epoch 3, Batch 400] loss: 1.856
[Epoch 3, Batch 600] loss: 1.804
[Epoch 3, Batch 600] loss: 1.804
[Epoch 4, Batch 200] loss: 1.743
[Epoch 4, Batch 200] loss: 1.743
[Epoch 4, Batch 400] loss: 1.704
[Epoch 4, Batch 400] loss: 1.704
[Epoch 4, Batch 600] loss: 1.668
[Epoch 4, Batch 600] loss: 1.668
[Epoch 5, Batch 200] loss: 1.611
[Epoch 5, Batch 200] loss: 1.611
[Epoch 5, Batch 400] loss: 1.579
[Epoch 5, Batch 400] loss: 1.579
[Epoch 5, Batch 600] loss: 1.575
[Epoch 5, Batch 600] loss: 1.575
[Epoch 6, Batch 200] loss: 1.528
[Epoch 6, 

In [7]:
# Evaluate on test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 61.85%


In [8]:
# Save model weights
Path('models').mkdir(parents=True, exist_ok=True)
PATH = './models/cifar_net.pth'
torch.save(net.state_dict(), PATH)
print('Saved model to', PATH)

Saved model to ./models/cifar_net.pth
